# Human Action Recognition with CNN

Installing package to downlaod dataset from Kaggle

In [ ]:
!pip install opendatasets

Importing the dataset from Kaggle

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/meetnagadia/human-action-recognition-har-dataset")
#2e344cb00471082503f5314871abf63d

In [ ]:
#import of numpy and pandas libraries, set of mixed precision
import numpy as np
import pandas as pd
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


Definition of model hyperparameters

In [ ]:
FINE_TUNING_EPOCHS = 8  #number of epochs for training
BATCH_SIZE = 32

#dimension in pixel of the images
image_height = 180
image_width = 300

Reading of input file (test/training)

In [ ]:
#path to the data files
#assign of a variable for the folder path with input files
main_path = "/content/human-action-recognition-har-dataset/Human Action Recognition"

#read data
test_df     = pd.read_csv("/content/human-action-recognition-har-dataset/Human Action Recognition/Testing_set.csv")
training_df = pd.read_csv("/content/human-action-recognition-har-dataset/Human Action Recognition/Training_set.csv")

Connecting Google Drive and copying my trained model on a new folder

In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/drive')
drive_location = '/content/drive/My Drive/Colab Notebooks/HAR'

shutil.copytree(drive_location,'./myInceptionModel')

Creation of training and validation data from training dataframe:


*   Validation set: 10% of original training file
*   Training set: 90% of original training file



In [ ]:

valid_df = training_df.groupby('label').apply(lambda x: x.sample(frac=0.10))
valid_df = valid_df.droplevel(0)
train_df = training_df.drop(valid_df.index)

In [ ]:
#print first five element of training dataset with label
training_df.head()

Definition of variables to print pie chart

In [ ]:
freq = training_df['label'].value_counts()
train_freq = train_df['label'].value_counts()
valid_freq = valid_df['label'].value_counts()


In [ ]:
#pie graph representation of class frequency, the dataset is balanced
freq.plot(kind='pie',figsize=(5,5),title='Human Activity',autopct='%1.1f%%',shadow = False,  fontsize=8);

# Pre-processing

In [ ]:
#importing deep learning libraries
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

In [ ]:
#data Augmentation
gen = ImageDataGenerator(rescale         = 1./255.,
                         horizontal_flip = True,
                         vertical_flip   = False,
                         zoom_range      = 0.07,
                         rotation_range  = 25,
                         brightness_range=[0.5,1.5]
                         )

train_generator = gen.flow_from_dataframe(
    train_df,
    directory = main_path + "/train",
    x_col = 'filename',
    y_col = 'label',
    color_mode = "rgb",
    target_size = (image_height, image_width),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 42)

In [ ]:
test_gen = ImageDataGenerator(rescale = 1./255.)

validation_generator = test_gen.flow_from_dataframe(
    valid_df,
    directory   = main_path + "/train",
    x_col       = 'filename',
    y_col       = 'label',
    color_mode  ="rgb",
    target_size = (image_height, image_width),
    class_mode  = "categorical",
    batch_size  = BATCH_SIZE,
    shuffle     = False,
    seed        = 42)

test_generator = test_gen.flow_from_dataframe(
    test_df,
    directory   = main_path + "/test",
    x_col       = 'filename',
    #y_col       = 'labels',
    color_mode  = "rgb",
    target_size = (image_height, image_width),
    class_mode = None,
    batch_size  = BATCH_SIZE,
    shuffle  = False)

In [ ]:
#save all training labels
a = train_generator.class_indices
class_names = list(a.keys())

In [ ]:
class_names

In [ ]:
#define function to print testing images with their label
import matplotlib.pyplot as plt

def plot_images(img, true_labels, predictions = None):
    plt.figure(figsize=[12, 18])
    for i in range(24):
        plt.subplot(6, 4, i+1)
        plt.imshow(img[i])
        plt.axis('off')
        if (predictions is not None):
            if true_labels is not None:
                plt.title("{}\n {} {:.1f}%".format(class_names[np.argmax(true_labels[i])], class_names[np.argmax(predictions[i])], 100 * np.max(predictions[i])))
            else:
                plt.title("{} {:.1f}%".format(class_names[np.argmax(predictions[i])], 100 * np.max(predictions[i])))
        else:
            if true_labels is not None:
                plt.title(class_names[np.argmax(true_labels[i])])

# Definition of the function necessary to create the confusion matrix

In [ ]:
#import libraries to show confusion matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix #slkearn module

def plot_confusion_matrix(y_true, y_pred, class_names, ax, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(
        cm,
        annot=True,
        square=True,
        xticklabels=class_names,
        yticklabels=class_names,
        fmt='d',
        cmap=plt.cm.Blues,
        cbar=False,
        ax=ax)
    ax.set_title(title, fontsize = 12)
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 45, ha = "right")
    ax.set_ylabel('True Label', fontsize= 10)
    ax.set_xlabel('Predicted Label', fontsize = 10)

# Definition Model

In [ ]:
#import tf module
from tensorflow.keras.models import Model

def create_model(base_model):

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation = 'relu')(x) #activation function relu
    x = Dropout(0.4)(x)
    x = Dense(64, activation = 'relu')(x)
    x = Dropout(0.2)(x)

    outputs = Dense(len(class_names), activation='softmax')(x) #activation function softmax

    model = Model(base_model.inputs, outputs)

    return model

In [ ]:
#specify how to train the model

def fit_model(model, base_model, epochs, fine_tune = 0):

    early = tf.keras.callbacks.EarlyStopping( patience = 10,
                                              min_delta = 0.001,
                                              restore_best_weights = True)

    base_model.trainable = True
    for layer in base_model.layers[:-fine_tune]:
        layer.trainable = False

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003,epsilon=1e-06),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


    history = model.fit(train_generator,
                        validation_data = validation_generator,
                        epochs = epochs,
                        callbacks = [early])

    return history



# InceptionResNetV2 model creation

In [ ]:
#pretrained model from Keras assigned to inception_base_model
inception_base_model = tf.keras.applications.InceptionResNetV2(
                     include_top = False,
                     weights = 'imagenet',
                     input_shape = (image_height, image_width, 3)
                     )

inception_model = create_model(inception_base_model) #model creation with definition model


In [ ]:
nr_base_layers = len(inception_base_model.layers) #parameter to do training
print("Inception base layers = ", nr_base_layers)

#effect model training and saves accuracy and loss results in history variable to do graphics
history = fit_model(inception_model, inception_base_model, epochs = FINE_TUNING_EPOCHS, fine_tune = int(nr_base_layers/4))

#save the trained model in folder myInceptionModel
inception_model.save('myInceptionModel')

Uploading model to Google Drive

In [ ]:
#upload the entire model saved in Google Drive folder
shutil.copytree('./myInceptionModel',drive_location)

Loading model from myInceptionModel

In [ ]:
inception_model = tf.keras.models.load_model('myInceptionModel')

In [ ]:
#estimate accuracy score on validation images
accuracy_score = inception_model.evaluate(validation_generator)

print(accuracy_score)
print("Accuracy: {:.2f}%".format(accuracy_score[1] * 100))
print("Loss: {:.3f}".format(accuracy_score[0]))


In [ ]:
#variables to print confusion matrix
validation_generator.reset()
inception_val_preds = inception_model.predict(validation_generator)
inception_val_pred_classes = np.argmax(inception_val_preds, axis=1)

In [ ]:
#effect test prediction
test_generator.reset()
inception_test_preds = inception_model.predict(test_generator)
inception_test_pred_classes = np.argmax(inception_test_preds, axis=1)

In [ ]:
#show test results
test_generator.reset()
x = next(test_generator)
plot_images(x, None, inception_test_preds)

#Confusion matrix

In [ ]:
#display confusion matrix of validation data
fig, (ax1) = plt.subplots(1, figsize=(10, 8))

true_classes = validation_generator.classes

plot_confusion_matrix(true_classes, inception_val_pred_classes, class_names, ax1, title = "Inception")

fig.suptitle("Confusion Matrix", fontsize = 12)
fig.tight_layout()
fig.subplots_adjust(top=0.91)
plt.show()